In [1]:
%matplotlib notebook  

In [2]:
!nvidia-smi

Wed Aug  4 06:44:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      Off  | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    36W / 250W |   5253MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-PCIE-40GB      Off  | 00000000:25:00.0 Off |                    0 |
| N/A   

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd

# Python 3 standard library
from pathlib import Path

from model.collectdata_kde_Ellipsoids import collect_t2kde_data

##  It takes 9 input features (poca centers + (A,B,C,D,E,F)) . 
from model.models_kde import TracksToKDE_Ellipsoids_DDplus as Model

# kde_loss_Ba is a Chi2 loss (go and look at it!)
from model.kde_loss_Ba import Loss

from model.training_kde import trainNet, select_gpu, Results
from model.plots import dual_train_plots, dual_train_plots_tracks_to_KDE, replace_in_ax

In [4]:
#This notebook should be functionally the same as RunModel_Tracks_to_KDE_DDplus_it29_SetVar_Architecture_Comparison_jgo.
#The only reason that notebook exists is to run model training in parallel. 

n_epochs = 100
# Size of batches
batch_size = 64 ## batch_size = 24 ---> 4763MiB / 12066MiB on Titan V
# How fast to learn
learning_rate = 1e-6
architectures = [[50]*11]
latentChannels = 4
model_names = ['50_nodes_exp_trainset_slow']

previous = ['jgo_files/02August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_50_nodes_sliceless_250_epochs_1e-05/02August2021_DDplus_loss_Ba_iter29_floatAll_SetVar_50_nodes_sliceless_250_epochs_1e-05_final.pyt']

# n_epochs = 50
# # Size of batches
# batch_size = 64 ## batch_size = 24 ---> 4763MiB / 12066MiB on Titan V
# # How fast to learn
# learning_rate = 1e-4
# architectures = [[5, 7, 10, 25, 50, 75, 100, 150, 250, 350, 500]]
# latentChannels = 4
# model_names = ['5to500_nodes_unstick_attpt']

# previous = ['jgo_files/20July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_it0pretrain_250_epochs_1e-05/20July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_it0pretrain_250_epochs_1e-05_final.pyt']



# architectures = [[5]*11,
#                  [10]*11,
#                  [15]*11
#                  [20]*11
#                  [25]*11
#                  [30]*11
#                  [50]*11
#                  [5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20],
#                  [5, 5, 5, 6, 7, 8, 10, 12, 14, 17, 20],
#                  [5, 6, 7, 8, 9, 11, 12, 14, 16, 17, 20]
#                  [5, 9, 14, 18, 23, 27, 32, 36, 41, 45, 50],
#                  [5, 13, 23, 31, 41, 49, 59, 67, 77, 85, 100],
#                  [5, 7, 10, 25, 50, 75, 100, 150, 250, 350, 500]]
# model_names = ['5_nodes',
#                '10_nodes',
#                '15_nodes',
#                '20_nodes',
#                '25_nodes',
#                '30_nodes',
#                '50_nodes',
#                '5to20_lin_inc_nodes',
#                '5to20_quadly_inc_nodes',
#                '5to20_exp_inc_nodes',
#                '5to50_nodes'
#                '5to500_exp_inc_nodes']

device = select_gpu(2)
##device = "cpu"

1 available GPUs (initially using device 0):
  0 A100-PCIE-40GB


In [5]:
# Data Import. If we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
# the dataset will overflow the GPU memory; device=device will allow the data to move back
# and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
# down performance by about 10%.  So comment out when not needed.

## Training dataset. You can put as many files here as desired.

#train_loader = collect_t2kde_data('/share/lazy/pv-finder/20k_evts_for_KDE_learning_200716.h5',
train_loader = collect_t2kde_data('/share/lazy/will/data/June30_2020_80k_1.h5', 
                                  '/share/lazy/will/data/June30_2020_80k_2.h5',
                                    '/share/lazy/will/data/June30_2020_80k_3.h5',
                                    '/share/lazy/will/data/June30_2020_80k_4.h5',
                                      '/share/lazy/will/data/June30_2020_80k_5.h5',
                                      '/share/lazy/will/data/June30_2020_80k_6.h5',
                                      '/share/lazy/will/data/June30_2020_80k_7.h5',
#     train_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                             batch_size=batch_size,
## if we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
## the dataset will overflow the GPU memory; device=device will allow the data to move back
## and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
## down performance by about 10%.  So comment out when not needed.
#                               device=device,
                           # slice = slice(None,10000)
                           )

# Validation dataset. You can slice to reduce the size.
## mds no separate validation set yet,

## For iter12, change slice(18000,None) to slice(10000,None)
## First, we'll see if this changes the validation cost significantly
##  Second, we will see if this reduces the validation cost fluctuations
val_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                            batch_size=batch_size,
#                                 device=device,
                            # slice = slice(10000,None)
                           )

Loading data...


/data/home/jgocain/.local/lib/python3.8/site-packages/awkward0/persist.py:509: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  schema = schema.tostring()


pocaMx.shape =  (80000,)
nEvts =  80000
len(pocaMx[0]) =  199
len(pocaMx[1]) =  25
len(pocaMx[2]) =  369
len(pocaMx[3]) =  143
len(pocaMx[4]) =  160
majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 199
 iEvt, nTrks =  1 25
 iEvt, nTrks =  2 369
 iEvt, nTrks =  3 143
 iEvt, nTrks =  4 160
 iEvt, nTrks =  5 260
 iEvt, nTrks =  6 237
 iEvt, nTrks =  7 327
 iEvt, nTrks =  8 178
 iEvt, nTrks =  9 106
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  0.00045611936
majorAxis[iTrk][1][0] =  -4.8292455e-05
majorAxis[iTrk][2][0] =  0.090019904
minorAxis_1[iTrk][0][0] =  -1.8602173
minorAxis_1[iTrk][1][0] =  -17.569641
minorAxis_1[iTrk][2][0] =  4.7891795e-08
minorAxis_2[iTrk][0][0] =  -17.569414
minorAxis_2[iTrk][1][0] =  1.8601931
minorAxis_2[iTrk][2][0] =  0.0900199
  
majorAxis[iTrk][0][0] =  0.002360258
majorAxis[iTrk][1][0] =  -0.007426616
majorAxis[iTrk][2][0] =  0.3710108


majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 196
 iEvt, nTrks =  1 35
 iEvt, nTrks =  2 265
 iEvt, nTrks =  3 128
 iEvt, nTrks =  4 124
 iEvt, nTrks =  5 122
 iEvt, nTrks =  6 300
 iEvt, nTrks =  7 179
 iEvt, nTrks =  8 243
 iEvt, nTrks =  9 112
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  -2.440992e-05
majorAxis[iTrk][1][0] =  -0.00038087237
majorAxis[iTrk][2][0] =  0.08211532
minorAxis_1[iTrk][0][0] =  -17.631672
minorAxis_1[iTrk][1][0] =  1.130005
minorAxis_1[iTrk][2][0] =  4.7444193e-08
minorAxis_2[iTrk][0][0] =  1.1299927
minorAxis_2[iTrk][1][0] =  17.631481
minorAxis_2[iTrk][2][0] =  0.08211532
  
majorAxis[iTrk][0][0] =  -0.02277395
majorAxis[iTrk][1][0] =  0.0014473597
majorAxis[iTrk][2][0] =  0.6347583
minorAxis_1[iTrk][0][0] =  -1.1205891
minorAxis_1[iTrk][1][0] =  -17.632273
minorAxis_1[iTrk][2][0] =  2.1543272e-09
minorAxis_2[iTrk][0][0] =  -17.6208

minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 204
 iEvt, nTrks =  1 21
 iEvt, nTrks =  2 226
 iEvt, nTrks =  3 138
 iEvt, nTrks =  4 147
 iEvt, nTrks =  5 226
 iEvt, nTrks =  6 315
 iEvt, nTrks =  7 298
 iEvt, nTrks =  8 113
 iEvt, nTrks =  9 53
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  0.0008031223
majorAxis[iTrk][1][0] =  0.0026818695
majorAxis[iTrk][2][0] =  0.2223914
minorAxis_1[iTrk][0][0] =  -16.925222
minorAxis_1[iTrk][1][0] =  5.0684876
minorAxis_1[iTrk][2][0] =  -2.4095305e-08
minorAxis_2[iTrk][0][0] =  5.0680866
minorAxis_2[iTrk][1][0] =  16.923882
minorAxis_2[iTrk][2][0] =  -0.2223914
  
majorAxis[iTrk][0][0] =  0.0024043417
majorAxis[iTrk][1][0] =  -0.0074925683
majorAxis[iTrk][2][0] =  0.37282085
minorAxis_1[iTrk][0][0] =  -16.822897
minorAxis_1[iTrk][1][0] =  -5.3984146
minorAxis_1[iTrk][2][0] =  -9.030704e-09
minorAxis_2[iTrk][0][0] =  -5.3972125
minorAxis_2[iTrk][1][0] =  16.81915
minorAxis_2[iTrk][2][0]

  nEvts =  80000
 iEvt, nTrks =  0 196
 iEvt, nTrks =  1 310
 iEvt, nTrks =  2 201
 iEvt, nTrks =  3 58
 iEvt, nTrks =  4 135
 iEvt, nTrks =  5 369
 iEvt, nTrks =  6 226
 iEvt, nTrks =  7 198
 iEvt, nTrks =  8 145
 iEvt, nTrks =  9 130
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  0.002647314
majorAxis[iTrk][1][0] =  0.008641847
majorAxis[iTrk][2][0] =  0.3995565
minorAxis_1[iTrk][0][0] =  16.89298
minorAxis_1[iTrk][1][0] =  -5.1749372
minorAxis_1[iTrk][2][0] =  -4.4746007e-09
minorAxis_2[iTrk][0][0] =  -5.173614
minorAxis_2[iTrk][1][0] =  -16.88866
minorAxis_2[iTrk][2][0] =  0.39955652
  
majorAxis[iTrk][0][0] =  2.229869e-06
majorAxis[iTrk][1][0] =  9.085482e-05
majorAxis[iTrk][2][0] =  0.04007104
minorAxis_1[iTrk][0][0] =  -17.662525
minorAxis_1[iTrk][1][0] =  0.43349507
minorAxis_1[iTrk][2][0] =  4.4459145e-07
minorAxis_2[iTrk][0][0] =  0.433494
minorAxis_2[iTrk][1][0] =  17.66248
minorAxis_2[iTrk][2][0] =  -0.040071033
  
majorAxis[iTrk][0][0] =  -0.0064935046
majorAxis[iTrk][1][0]

In [ ]:
destinations = []

for i in range(len(model_names)):
    modelstarttime = time.strftime("%d/%m/%Y %H:%M:%S")
    if previous[i] != 'None':
        folder = time.strftime('%d%B%Y') + '_DDplus_loss_Ba_iter29_floatAll_SetVar_' + model_names[i] + '_' + str(n_epochs) + '_epochs_' + str(learning_rate)
    else:
        folder = time.strftime('%d%B%Y') + '_DDplus_loss_Ba_iter29_floatAll_SetVar_' + model_names[i] + '_nopretrain_' + str(n_epochs) + '_epochs_' + str(learning_rate)
    name   = folder

    # Make an output folder named "name" (change if you want)

    ## Special instructions for those working on goofy at UC
    ## Please be very careful to make sure that your folder
    ## does not live in a subdirectory of your home directory
    ## this disk has very little capacity. Instead, use 
    ## a subdirectory in /share/lazy with a symbolic link to
    ## it in this (the notebooks) subdirectory
    folder = 'jgo_files/' + folder
    output = Path(folder)
    output.mkdir(exist_ok=True)
    
    #Data Frame for storing learning results
    results = pd.DataFrame([], columns=Results._fields)
    
    #Building the Model per settings written in 'architectures' array
    nOut1, nOut2, nOut3, nOut4, nOut5, nOut6, nOut7, nOut8, nOut9, nOut10, nOut11 = architectures[i]
    model = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11)
    
    #Defining Loss Function
    loss = Loss(epsilon=3e-6)
    
    model_dict = model.state_dict()
    #Transfering Model to GPU (or CPU, if chosen)
    model = model.to(device)
    
    updated_dict = model_dict
    ##print("updated_dict = ",updated_dict)
    ## when starting "ab initio", reduce biases as the bias gets summed for each track
    ## contributing to the predicted KDE
    updated_dict["layer1.bias"] = 0.005*model_dict["layer1.bias"]
    updated_dict["layer2.bias"] = 0.005*model_dict["layer2.bias"]
    updated_dict["layer3.bias"] = 0.005*model_dict["layer3.bias"]
    updated_dict["layer4.bias"] = 0.005*model_dict["layer4.bias"]
    updated_dict["layer5.bias"] = 0.005*model_dict["layer5.bias"]
    updated_dict["layer6.bias"] = 0.005*model_dict["layer6.bias"]
    updated_dict["layer7.bias"] = 0.005*model_dict["layer7.bias"]
    updated_dict["layer8.bias"] = 0.005*model_dict["layer8.bias"]
    updated_dict["layer9.bias"] = 0.005*model_dict["layer9.bias"]
    updated_dict["layer10.bias"] = 0.005*model_dict["layer10.bias"]
    updated_dict["layer11.bias"] = 0.005*model_dict["layer11.bias"]
    model.load_state_dict(updated_dict,strict=False)

    model_dict = model.state_dict()
    
    print(model_names[i])
    if previous[i] != 'None':
        pretrained_dict = torch.load(previous[i])
        print(" ")
        print("  for pretrained_dict")
        index = 0
        for k,v in pretrained_dict.items():
            print("index, k =  ",index,"  ",k)
            index = index+1
        ##print("model_dict instantiated")
        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        print("pretrained_dict iterated")
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        ##
        #   when starting from a model with a fully connected last layer rather than a convolutional layer
        # 3. load the new state dict
        #   need to use strict=False as the two models state model attributes do not agree exactly
        #   see https://pytorch.org/docs/master/_modules/torch/nn/modules/module.html#Module.load_state_dict

        model.load_state_dict(pretrained_dict,strict=False)
    
    #Defining Optimizer (Reminder to ask about that)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Data Import. If we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
    # the dataset will overflow the GPU memory; device=device will allow the data to move back
    # and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
    # down performance by about 10%.  So comment out when not needed.

    
    for result in trainNet(model, optimizer, loss, train_loader, val_loader, n_epochs, epoch_start=len(results), notebook=True):
        
        #Record Epoch Results
        results = results.append(pd.Series(result._asdict()), ignore_index=True)

        # Save each model state dictionary
        torch.save(model.state_dict(), output / f'{name}_{result.epoch}.pyt')
    
    #Save final point in model
    torch.save(model.state_dict(), output / f'{name}_final.pyt')
    print('Final Model State Saved to', output / f'{name}_final.pyt')
    
    results.to_hdf(output / f'{name}_stats.hdf5', 'results')
    print('Model Statistics Saved to', output / f'{name}_stats.hdf5')
    
    destinations.append(output / f'{name}_stats.hdf5')
    
    readme = open(output / f'{name}_README.txt', 'w')
    readme.write('Model Run using RunModel_Tracks_to_KDE_DDplus_it29_SetVar_Architecture_Comparison_jgo_copy \n')
    readme.write('Model Run from ' + modelstarttime + ' to ' + time.strftime("%d/%m/%Y %H:%M:%S"))
    readme.write('\n')
    readme.write('Model Architecture:' + ' \n')
    childindex = 0
    for child in model.children():
        readme.write(str(childindex) + '    ' + str(child) + ' \n')
        childindex = childindex+1
    readme.write('\n')
    readme.write('No Layers Frozen' + ' \n')
    readme.write('\n')
    readme.write('Pre-training Used: ' + previous[i] + ' \n')
    readme.write('\n')
    readme.write('Training Data Used:' + ' \n')
    for trainfile in ['/share/lazy/will/data/June30_2020_80k_1.h5', 
                                  '/share/lazy/will/data/June30_2020_80k_2.h5',
                                    '/share/lazy/will/data/June30_2020_80k_3.h5',
                                    '/share/lazy/will/data/June30_2020_80k_4.h5',
                                      '/share/lazy/will/data/June30_2020_80k_5.h5',
                                      '/share/lazy/will/data/June30_2020_80k_6.h5',
                                      '/share/lazy/will/data/June30_2020_80k_7.h5']:
        readme.write(trainfile + ' \n')
    readme.write('No Training Slice Used \n')
    readme.write('\n')
    readme.write('No Validation Data Used' + ' \n')
    for testfile in ['dataAA/20K_POCA_kernel_evts_200926.h5']:
        readme.write(testfile + '\n')
    readme.write('\n')
    readme.write('Batch Size:' + str(batch_size) + ' \n')
    readme.write('\n')
    readme.write('Loss Function: Ba' + ' \n')
    readme.write('\n')
    readme.write('Optimizer Used: Adam' + ' \n')
    readme.write('\n')
    readme.write('Latent Channels' + str(latentChannels) + ' \n')
    readme.close()
    print('Model Settings Saved to', output / f'{name}_README.txt')
    
    
    #

50_nodes_exp_trainset_slow
 
  for pretrained_dict
index, k =   0    layer1.weight
index, k =   1    layer1.bias
index, k =   2    layer2.weight
index, k =   3    layer2.bias
index, k =   4    layer3.weight
index, k =   5    layer3.bias
index, k =   6    layer4.weight
index, k =   7    layer4.bias
index, k =   8    layer5.weight
index, k =   9    layer5.bias
index, k =   10    layer6.weight
index, k =   11    layer6.bias
index, k =   12    layer7.weight
index, k =   13    layer7.bias
index, k =   14    layer8.weight
index, k =   15    layer8.bias
index, k =   16    layer9.weight
index, k =   17    layer9.bias
index, k =   18    layer10.weight
index, k =   19    layer10.bias
index, k =   20    layer11.weight
index, k =   21    layer11.bias
index, k =   22    layer12new.weight
index, k =   23    layer12new.bias
index, k =   24    conv1.weight
index, k =   25    conv1.bias
index, k =   26    conv2.weight
index, k =   27    conv2.bias
index, k =   28    fc1.weight
index, k =   29    fc1.bi

Epochs:   0%|                                                           | 0/100 [00:00<?, ?it/s, train=start, …

Number of batches: train = 8750, val = 313


Training:   0%|                                                                   | 0/8750 [00:00<?, ?it/s, tr…

Epoch 0: train=4.51833, val=3.88759, took 1045.1 s


...

Training:   0%|                                                                          | 0/8750 [00:00<?, ?i…

Epoch 97: train=4.30893, val=3.65542, took 835.11 s


Training:   0%|                                                                          | 0/8750 [00:00<?, ?i…

Epoch 98: train=4.30807, val=3.64293, took 831.64 s


Training:   0%|                                                                          | 0/8750 [00:00<?, ?i…

In [ ]:
#Print File destinations
stringer = ''
for file in destinations:
    stringer = stringer + '\'' + name + '\', '
print(stringer)

In [ ]:
print('Full Run Time')

In [ ]:
quit()
